In [1]:
#transfection efficiency test setup

In [2]:
import numpy as np
import pandas as pd
from pint import UnitRegistry
u = UnitRegistry()
u.define('cells = [cells] = cells')

In [25]:
# scaling protocols to fit in half deep well (200 uL)
#Reagent 007 (R007): not needed

#Freestyle Max (FM): smallest protocol is for 30 mL 
fm_original_volume = 30 * u.mL
fm_volume = 200 * u.uL
fm_scaling = (fm_original_volume/fm_volume).to_reduced_units().magnitude
fm_cells = 3E7 * u.cells/fm_scaling
fm_plasmid = 37.5 * u.ug/fm_scaling
fm_reagent = 37.5 * u.uL/fm_scaling
fm_plasmid_reagent_complex_volume = 0.6*u.mL/fm_scaling
print(f"Results for FM: {fm_volume:~P}, {fm_cells:,~Pg}, {fm_plasmid:~Pg}, {fm_reagent:~Pg}, {fm_plasmid_reagent_complex_volume.to(u.uL):~Pg}")

#PEIpro (PP): smallest protocol is for 30 mL
pp_original_volume = 30 * u.mL
pp_volume = 200 * u.uL
pp_scaling = (pp_original_volume/pp_volume).to_reduced_units().magnitude
pp_cells = 3E7 * u.cells/pp_scaling
pp_plasmid = 30 * u.ug/pp_scaling
pp_reagent = 90 * u.uL/pp_scaling
pp_plasmid_reagent_complex_volume = 1.5*u.mL/pp_scaling
print(f"Results for PP: {pp_volume:~P}, {pp_cells:,~Pg}, {pp_plasmid:~Pg}, {pp_reagent:~Pg}, {pp_plasmid_reagent_complex_volume.to(u.uL):~Pg}")

#FectoPRO (FP): protocol is per 1 mL
fp_original_volume = 1 * u.mL
fp_volume = 200 * u.uL
fp_scaling = (fp_original_volume/fp_volume).to_reduced_units().magnitude
fp_cells = 1E6 * u.cells/fp_scaling
fp_plasmid = 0.5 * u.ug/fp_scaling
fp_reagent = 1 * u.uL/fp_scaling
fp_plasmid_reagent_complex_volume = 0.1*u.mL/fp_scaling
print(f"Results for FP:\n\tTotal volume: {fp_volume:~P}\n\tNumber of cells: {fp_cells:,~Pg}\n\tAmount of DNA: {fp_plasmid:~Pg}\n\tVolume of reagent: {fp_reagent:~Pg}\n\tVolume of DNA+reagent: {fp_plasmid_reagent_complex_volume.to(u.uL):~Pg}")


Results for FM: 200 ul, 200,000 cells, 0.25 ug, 0.25 ul, 4 ul
Results for PP: 200 ul, 200,000 cells, 0.2 ug, 0.6 ul, 10 ul
Results for FP:
	Total volume: 200 ul
	Number of cells: 200,000 cells
	Amount of DNA: 0.1 ug
	Volume of reagent: 0.2 ul
	Volume of DNA+reagent: 20 ul


# Manufacturers protocol scaled to 200 uL
## Reagent 007 (R007):
    Dissolve in sterile deionized water: 1mg/mL = 4x stock
    Dillute 4 times to get 1x working solution
    
    1) seed cells in wells
        Media: ab free glutamax supplemented media
        Cells: 1.5E5
        Volume: 100 uL, ideally done by taking 50 uL 3E6 cells/mL and adding 50 uL fresh.
        (Taking our x3 cell concentration into account, this would mean having to have 9E6 cells/mL. Thats a bit too dense. Will just use 100 uL cells)
    2) form complexes: Warning: If DNA is in TE buffer, then it must be dilluted so that less than 1.5 uL TE buffer is used.
        DNA: 0.5 ug DNA, final volume 5 uL (water)
        MIX: add 10 uL R007 WS
        Mix immidiatly by gently pipetting 5-6 times. DO NOT VORTEX.
        wait 5 min RT
    3) add all the 15 uL R007+DNA to cells slowly while shaking cells gently
    4) incubate cells (on shaker....hmm) for 1.5 hours
    5) Add 100 uL fresh media
## Freestyle Max (FM):
    1) Seed cells:
        Media: 
        Cells: 2E5
        Volume: 192 uL
    2) Form complexes
        DNA: 0.25 ug DNA, final volume 4 uL (OptiPRO SFM)
        REAGENT:  (web vs pdf protocol differences: Saranya will test. Writing pdf version here)
            Gently invert Freestyle Max reagent 4 times (no vortex)
            0.25 uL FM + 3.75 uL OptiPRO SFM, Mix gently, Wait 5 min RT (NOT LONGER THAN 5 min)
        Mix DNA+REAGENT
        Wait 20-30 minutes at RT
    3) Add the 8 uL complex to the 192 uL media for a total of 200 uL and 1E6 cells/mL
## PEIpro (PP)
    1) Seed cells
        Cells: 2E5
        Volume: 180 uL
    2) form complexes:
        DNA: 0.2 ug DNA, final volume of 10 uL (serum free medium, no pluronic f-68, no antibiotics)
            Vortex gently
        REAGENT: Vortex PEIpro for 5 sec and spin down
            Dilute 0.6 uL PEIpro in the same kind of media as the DNA. Final volume 10 uL
        Vortex gently and immidiatly add the PEIPro to the DNA and votex immidiatly
        Wait 15 min RT
    3) Add the 20 uL complex to the 180 uL media
## FectoPRO (FP)
    1) Seed 
        Cells: 2E5
        Volume: 180 uL
    2) Complex
        DNA: 0.1 ug DNA, final volume 20 uL
        REAGENT: vortex FP for 5 sec and spin down
        Add 0.2 uL reagent to DNA (homogenize...vortex?)
        Wait 10 min RT
    3)  Add to cells (homogenize)

In [336]:
medias = ['CD_CHO', 'Freestyle_EM_CHO']
recommended_cell = {'R007': 1.5E5*u.cells, 'FM': 2E5*u.cells, 'PP': 2E5*u.cells, 'FP': 2E5*u.cells} # total cell number in a well based on reagent and maybe media
recommended_cell_volume = {'R007': 100*u.uL, 'FM': 192*u.uL, 'PP': 180*u.uL, 'FP': 180*u.uL}
recommended_dna = {'R007': 0.5*u.ug, 'FM': 0.25*u.ug, 'PP': 0.2*u.ug, 'FP': 0.1 * u.ug}
recommended_dna_volume = {'R007': 5*u.uL, 'FM': 4*u.uL, 'PP': 10*u.uL, 'FP': 20 * u.uL}
recommended_reagent = {'R007': 10*u.uL, 'FM': 0.25*u.uL, 'PP': 0.6*u.uL, 'FP': 0.2*u.uL}
reagents = ['R007', 'FM', 'PP', 'FP']

dna_levels = [0.5, 1, 2] # dna amount multiplication factors
cell_density_levels = [1, 2, 3] # reagent to dna ratio multiplication factors

replicates = 3

In [354]:
table = []
for reagent in reagents:
    for media in medias:
        for dna_level in dna_levels:
            for cdl in cell_density_levels:
                for rep in range(replicates):
                    #print(f"{reagent}\t{media}\t{recommended_cell[reagent]:,~Pg}\t{recommended_dna[reagent]*dna_level:~Pg}\t{recommended_reagent[reagent]*dna_level*ratio:~Pg}")
                    table.append([rep+1, reagent,media,recommended_cell[reagent]*cdl,(recommended_dna[reagent]*dna_level).to_compact(),recommended_reagent[reagent].to_compact()])

In [355]:
# number of 96 well plates
np.ceil(len(table)/96)

3.0

In [362]:
df = pd.DataFrame(table)
df.columns = ['Rep', 'Reagent', 'Media', 'Cells per well', 'DNA per well', "Reagent per well"]
df['DNA:Reagent ratio'] =  df.apply(lambda row: f"1:{1/(row['DNA per well'].to('ug')/row['Reagent per well'].to('uL')).magnitude:g}", axis=1)
df['Initial cell volume'] = df.apply(lambda row: recommended_cell_volume[row['Reagent']], axis=1)
df['Initial cell concentration'] =  df.apply(lambda row: (row['Cells per well']/row['Initial cell volume']).to('cells/mL'), axis=1)
df['Final cell concentration'] =  df.apply(lambda row: (row['Cells per well']/(200*u.uL)).to('cells/mL'), axis=1)
df['DNA volume'] = df.apply(lambda row: recommended_dna_volume[row['Reagent']].to_compact(), axis=1)
df['Initial DNA concentration'] = df.apply(lambda row: (row['DNA per well']/row['DNA volume']).to("ng/uL"), axis=1)
ign_columns = ['Rep', 'Reagent', 'Media', 'DNA:Reagent ratio']

df[df['Rep']==1].style.format({column:"{:~,gP}" for column in df.columns if column not in ign_columns})

,Rep,Reagent,Media,Cells per well,DNA per well,Reagent per well,DNA:Reagent ratio,Initial cell volume,Initial cell concentration,Final cell concentration,DNA volume,Initial DNA concentration
0,1,R007,CD_CHO,"150,000 cells",0.25 ug,10 ul,1:40,100 ul,1.5e+06 cells/ml,"750,000 cells/ml",5 ul,50 ng/ul
3,1,R007,CD_CHO,"300,000 cells",0.25 ug,10 ul,1:40,100 ul,3e+06 cells/ml,1.5e+06 cells/ml,5 ul,50 ng/ul
6,1,R007,CD_CHO,"450,000 cells",0.25 ug,10 ul,1:40,100 ul,4.5e+06 cells/ml,2.25e+06 cells/ml,5 ul,50 ng/ul
9,1,R007,CD_CHO,"150,000 cells",0.5 ug,10 ul,1:20,100 ul,1.5e+06 cells/ml,"750,000 cells/ml",5 ul,100 ng/ul
12,1,R007,CD_CHO,"300,000 cells",0.5 ug,10 ul,1:20,100 ul,3e+06 cells/ml,1.5e+06 cells/ml,5 ul,100 ng/ul
15,1,R007,CD_CHO,"450,000 cells",0.5 ug,10 ul,1:20,100 ul,4.5e+06 cells/ml,2.25e+06 cells/ml,5 ul,100 ng/ul
18,1,R007,CD_CHO,"150,000 cells",1 ug,10 ul,1:10,100 ul,1.5e+06 cells/ml,"750,000 cells/ml",5 ul,200 ng/ul
21,1,R007,CD_CHO,"300,000 cells",1 ug,10 ul,1:10,100 ul,3e+06 cells/ml,1.5e+06 cells/ml,5 ul,200 ng/ul
24,1,R007,CD_CHO,"450,000 cells",1 ug,10 ul,1:10,100 ul,4.5e+06 cells/ml,2.25e+06 cells/ml,5 ul,200 ng/ul
27,1,R007,Freestyle_EM_CHO,"150,000 cells",0.25 ug,10 ul,1:40,100 ul,1.5e+06 cells/ml,"750,000 cells/ml",5 ul,50 ng/ul


# Protocol introduction

In [387]:
print("We will use 1 plate per transfection reagent.")
print("Use B2->G10 for the experiment and B11-G12 for WT controls (12 wells, 6 per media)")
print("That means no edge wells will be used. Fill edge wells with 200 uL media")
# please add benchling ids
print(f"Reagents:\n\tCHO-SCells\n\tGFP plasmid\n\tHoects\n\tPI\n\tTransfection reagents ({reagents})\n\tmedias ({medias})")
print("Equipment:\n\thalf deel well plates (4)\n\t96 well plates (4)\n\tshake flasks (2)\n\ttubes (?)\n\tCeligo\n\tElectronic pipettes")

max_cell_concentration = df['Initial cell concentration'].max()
needed_cells_per_media = df.groupby('Media')['Cells per well'].sum().iloc[0]
print("\n\n")
print("Protocol:")
print(f"Grow CHO-S in the following media: {medias}")
print(f"The day before transfection (t=-1) you should passage cells for each media to ~{max_cell_concentration/2:~gP} in >{needed_cells_per_media/(max_cell_concentration/2):~gP}")

We will use 1 plate per transfection reagent.
Use B2->G10 for the experiment and B11-G12 for WT controls (12 wells, 6 per media)
That means no edge wells will be used. Fill edge wells with 200 uL media
Reagents:
	CHO-SCells
	GFP plasmid
	Hoects
	PI
	Transfection reagents (['R007', 'FM', 'PP', 'FP'])
	medias (['CD_CHO', 'Freestyle_EM_CHO'])
Equipment:
	half deel well plates (4)
	96 well plates (4)
	shake flasks (2)
	tubes (?)
	Celigo
	Electronic pipettes



Protocol:
Grow CHO-S in the following media: ['CD_CHO', 'Freestyle_EM_CHO']
The day before transfection (t=-1) you should passage cells for each media to ~2.25e+06 cells/ml in >18 ml


In [388]:
# cell seed calculator
measured_cell_density = {
    'CD_CHO': 1E7 * (u.cells/u.mL),
    'Freestyle_EM_CHO': 5E6 * (u.cells/u.mL)
}

extra_factor = 1.5

for media in medias:
    print(media)
    suggested_amount_of_cells = needed_cells_per_media*extra_factor
    print(f"\tYou need ~{suggested_amount_of_cells:~gP} and it should be in {(suggested_amount_of_cells/(max_cell_concentration/2)).to_compact():~gP} of {media}")
    volume_to_spin = suggested_amount_of_cells/measured_cell_density[media]
    print(f"\tSo spin down ~{volume_to_spin.to_compact():~gP} and resuspend in {(suggested_amount_of_cells/(max_cell_concentration/2)).to_compact():~gP} {media}")

CD_CHO
	You need ~6.075e+07 cells and it should be in 27 ml of CD_CHO
	So spin down ~6.075 ml and resuspend in 27 ml CD_CHO
Freestyle_EM_CHO
	You need ~6.075e+07 cells and it should be in 27 ml of Freestyle_EM_CHO
	So spin down ~12.15 ml and resuspend in 27 ml Freestyle_EM_CHO


# R007 protocol

In [402]:
reagent = "R007"
print(f"Detailed steps for {reagent}")
r007_df = df.query('Reagent == @reagent')
wells_needed = len(r007_df)
print(f"{wells_needed} wells in total spread over {len(medias)} medias, {len(dna_levels)} levels of DNA and {len(cell_density_levels)} levels of cell density for a total of {len(medias) * len(dna_levels) * len(cell_density_levels)} different setups and {replicates} replicates")

Detailed steps for R007
54 wells in total spread over 2 medias, 3 levels of DNA and 3 levels of cell density for a total of 18 different setups and 3 replicates


In [395]:
print("\tDNA:") # we have 3 levels of DNA
stock_dna_concentration = 1 * (u.ug/u.uL)
max_needed_conc = r007_df['Initial DNA concentration'].max()
volume_needed_per_well = recommended_dna_volume[reagent]


	DNA:


5 <Unit('microliter')>

In [371]:
print("\tCells:") # We have 3 levels of cell concentrations and we will use 1 cell stock at "max concentration" and then dilute it for the others
cells_needed = recommended_cell[reagent]*wells_calced_for


cell_concentration = (recommended_cell[reagent]/(50*u.uL)).to("cells/mL")
cell_volume = cells_needed/cell_concentration



print(f"\t\t{len(cell_density_levels)} levels of cell concentration. ")
#Take a 15 mL falcon tube and\n\t\tAdd {cell_volume.to_compact():~gP} cells ({cells_needed:gP} at {cell_concentration:~gP})")
print(f"\t\tAdd {(50*u.uL*wells_calced_for).to_compact():~gP} fresh media")

dna_max_amount_per_well = recommended_dna['R007']*dna_levels[-1]
#dna_volume_per_well = 
print(f"\tDNA:")

print("\tB11+B12: WT with no GFP or vialibility stain: Used to set gate thresholds on Celigo.")
print("\tC11+C12: WT with hoects: Used to ensure GFP/PI gate doesnt detect hoects.")
print("\tD11+D12: WT with PI: Used to ensure GFP/hoects gate doesnt detect PI.")
print("\tE11+E12: Transfected with no viability stain: Used to ensure PI/hoects gate doesnt detect GFP.")
print("\tF11+F12: WT with vialibility stain: Used to check WT viability")
print("\tG11+G12: Mock transfected recommended levels with vialibility stain: Used to contral against GFP toxicity")

Detailed steps for R007
DNA:
Total wells used for calcs: 60 (54 wells actually needed)
Prepare:
	Cells:
		3 levels of cell concentration. 
		Add 3 ml fresh media
	DNA:
	B11+B12: WT with no GFP or vialibility stain: Used to set gate thresholds on Celigo.
	C11+C12: WT with hoects: Used to ensure GFP/PI gate doesnt detect hoects.
	D11+D12: WT with PI: Used to ensure GFP/hoects gate doesnt detect PI.
	E11+E12: Transfected with no viability stain: Used to ensure PI/hoects gate doesnt detect GFP.
	F11+F12: WT with vialibility stain: Used to check WT viability
	G11+G12: Mock transfected recommended levels with vialibility stain: Used to contral against GFP toxicity


100800000.0 <Unit('cells')>

3.000000000000001 <Unit('milliliter')>